### Import Libraries

In [1]:
import pandas as pd

### Load Raw Data

In [ ]:
employee_survey_data = pd.read_csv('../data/employee_survey_data.csv')

general_data = pd.read_csv('../data/general_data.csv')

manager_survey_data = pd.read_csv('../data/manager_survey_data.csv')

in_time = pd.read_csv('../data/in_time.csv').rename(columns={'Unnamed: 0': 'EmployeeID'}).set_index('EmployeeID')

out_time = pd.read_csv('../data/out_time.csv').rename(columns={'Unnamed: 0': 'EmployeeID'}).set_index('EmployeeID')


In [9]:
in_time = in_time.apply(pd.to_datetime)

out_time = out_time.apply(pd.to_datetime)

actual_working_hours = out_time - in_time

working_hours_float = actual_working_hours.apply(lambda x: x.dt.total_seconds() / 3600)

mean_hours = working_hours_float.mean(axis=1)

overtime_days = (working_hours_float > 8).sum(axis=1)

absent_days = in_time.isnull().sum(axis=1)

time_features = pd.DataFrame({
    'EmployeeID': mean_hours.index,
    'Mean_Work_Hours': mean_hours.values,
    'Overtime_Days_Count': overtime_days.values,
    'Absent_Days_Count': absent_days.values
})

print(time_features.head())

   EmployeeID  Mean_Work_Hours  Overtime_Days_Count  Absent_Days_Count
0           1         7.373651                    0                 29
1           2         7.718969                   42                 25
2           3         7.013240                    0                 19
3           4         7.193678                    0                 26
4           5         8.006175                  115                 16


In [41]:
manager_survey_data

,EmployeeID,JobInvolvement,PerformanceRating
0,1,3,3
1,2,2,4
2,3,3,3
3,4,2,3
4,5,3,3
...,...,...,...
4405,4406,3,3
4406,4407,2,3
4407,4408,3,4
4408,4409,2,3
